In [152]:
# Setup the Jupyter version of Dash

import warnings
# ignoring deprecation warnings for jupyterdash
warnings.filterwarnings(
    "ignore",
    message=".*JupyterDash is deprecated.*",
    category=UserWarning,
)

from dash import dcc, html, dash_table, no_update
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import base64
import pandas as pd

from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Gaccurser"
# connecting to animal shelter db
shelter = AnimalShelter(username, password, host="localhost", port=27017)

# loading data into dataframe
df = pd.DataFrame.from_records(shelter.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
# creating dash app instance
app = JupyterDash(__name__)

# preparing logo image for layout
encoded_image = base64.b64encode(
    open('Grazioso Salvare Logo.png','rb').read()
).decode()

# setting up app layout
app.layout = html.Div([
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'100px'})),
    html.Center(html.H1('CS-340 Dashboard – Name')),
    html.Hr(),

    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label':'Reset Filters','value':'ALL'},
                {'label':'Water Rescue','value':'WR'},
                {'label':'Mountain/Wilderness','value':'MWR'},
                {'label':'Disaster/Tracking','value':'DRIT'}
            ],
            value='ALL',
            labelStyle={'display':'block','margin':'5px'}
        )
    ], style={'width':'40%','margin':'auto'}),
    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name":c,"id":c} for c in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        page_action="native",
        page_size=10,
        style_table={'overflowX':'auto'}
    ),
    html.Hr(),

    html.Div(style={'display':'flex','gap':'20px'}, children=[
        html.Div(id='graph-id', style={'flex':1}),
        html.Div(id='map-id',   style={'flex':1})
    ])
])

#############################################
# Controller: update table based on filter
#############################################
@app.callback(
    Output('datatable-id','data'),
    Input('filter-type','value')
)
def update_table(filter_type, **kwargs):
    # filtering data based on selected filter
    age_cond = {'age_upon_outcome_in_weeks': {'$lte': 104}}
    if filter_type == 'WR':
        query = {'$and': [
            {'sex_upon_outcome':'Intact Female'},
            {'breed':{'$in':[
                'Labrador Retriever Mix','Chesa Bay Retr Mix',
                'Newfoundland Mix','Newfoundland/Labrador Retriever',
                'Newfoundland/Australian Cattle Dog',
                'Newfoundland/Great Pyrenees'
            ]}},
            age_cond
        ]}
    elif filter_type == 'MWR':
        query = {'$and': [
            {'sex_upon_outcome':'Intact Male'},
            {'breed':{'$in':[
                'German Shepherd','Alaskan Malamute',
                'Old English Sheepdog','Rottweiler','Siberian Husky'
            ]}},
            age_cond
        ]}
    elif filter_type == 'DRIT':
        query = {'$and': [
            {'sex_upon_outcome':'Intact Male'},
            {'breed':{'$in':[
                'Doberman Pinscher','German Shepherd',
                'Golden Retriever','Bloodhound','Rottweiler'
            ]}},
            age_cond
        ]}
    else:
        query = {}

    dff = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

#############################################
# Controller: breed distribution pie chart
#############################################
@app.callback(
    Output('graph-id','children'),
    Input('datatable-id','derived_virtual_data')
)
def update_graph(viewData, **kwargs):
    if not viewData:
        return no_update
    # plotting breed distribution pie chart
    dff = pd.DataFrame(viewData)
    counts = dff['breed'].value_counts()
    fig = px.pie(names=counts.index, values=counts.values,
                 title='Breed Distribution', width=400, height=400)
    return dcc.Graph(figure=fig)

#############################################
# Controller: geolocation map of filtered rows
#############################################
@app.callback(
    Output('map-id','children'),
    Input('datatable-id','derived_virtual_data')
)
def update_map(viewData, **kwargs):
    if not viewData:
        return no_update
    # mapping pet locations on leaflet map
    dff = pd.DataFrame(viewData)
    markers = []
    for _, row in dff.iterrows():
        lat, lon = row['location_lat'], row['location_long']
        popup = html.Div([
            html.B(row['breed']), html.Br(),
            f"Sex: {row['sex_upon_outcome']}", html.Br(),
            f"Age: {row['age_upon_outcome']}"
        ])
        markers.append(dl.Marker(position=[lat, lon], children=[dl.Popup(popup)]))

    return dl.Map(
        children=[dl.TileLayer()] + markers,
        center=[30.2672, -97.7431], zoom=10,
        style={'width':'100%','height':'500px'}
    )

# launch in the notebook
if __name__ == '__main__':
    # launching the dash app
    app.run(debug=False)